# CUNY DATA 612 Project 1 | Global Baseline Predictors and RMSE
Ananya Punyatoya
06/07/2020

# Intoduction 
1. I have taken a relatively smaller and dense data set unlike real world project.
2. Each rows describes a particular user and each colums describes a particular book; Each cell represents a rating provided by an user for a particular book 
3. User bias and book bias are calculated for each user and each movie 
4. predicted rating = raw_avg(training data set only) + user_bias + book_bias
5. Used RMSE(root mean square error) to find the efficiencies of the model 

## importing Required library for building recomendation system 

In [122]:
import pandas as pd
import numpy as np


## Reading data from an excel file 

In [123]:
book_data = pd.read_excel(r"/Users/ananyap/Downloads/book_data.xlsx", encoding = 'unicode_escape',index_col=0 )
book_data = book_data.replace('?',np.nan)
book_data = book_data.values
book_data

array([[  5.,  nan,   4.,  nan,   4.],
       [  4.,   3.,   5.,   3.,   4.],
       [  4.,   2.,  nan,  nan,   3.],
       [  2.,   2.,   3.,   1.,   2.],
       [  4.,  nan,   5.,   4.,   5.],
       [  4.,   2.,   5.,   4.,   4.]])

## Spliting data into test and train set 

In [124]:
#creating index for tran and test data
pct_test = 0.2
n_test = int(round(pct_test * np.count_nonzero(~np.isnan(book_data)), 0))
non_na_index = np.argwhere(~np.isnan(book_data))
seq = np.arange(len(non_na_index[:, 0]))
test_index = np.random.choice(seq, n_test, replace = False)
train_index= np.delete(seq, test_ind)


In [125]:
#split  data set  into test and train
train = np.copy(book_data)
train[non_na_index[test_index, 0], non_na_ind[test_index, 1]] = np.nan
test = np.copy(book_data)
test[non_na_index[train_ind, 0], non_na_index[train_ind, 1]] = np.nan


In [126]:
train

array([[  5.,  nan,   4.,  nan,   4.],
       [  4.,  nan,   5.,   3.,   4.],
       [  4.,   2.,  nan,  nan,   3.],
       [  2.,   2.,   3.,   1.,   2.],
       [ nan,  nan,  nan,   4.,  nan],
       [  4.,   2.,  nan,   4.,   4.]])

## converting numpy array to pandas  data Frame 

In [156]:
train = pd.DataFrame(train,index=['p1','p2','p3','p4','p5','p6'],columns=['B1','B2','B3','B4','B5'])
train

,B1,B2,B3,B4,B5
p1,5.0,NaN,4.0,NaN,4.0
p2,4.0,NaN,5.0,3.0,4.0
p3,4.0,2.0,NaN,NaN,3.0
p4,2.0,2.0,3.0,1.0,2.0
p5,NaN,NaN,NaN,4.0,NaN
p6,4.0,2.0,NaN,4.0,4.0


In [157]:
test

,B1,B2,B3,B4,B5
p1,NaN,NaN,NaN,NaN,NaN
p2,NaN,3.0,5.0,NaN,NaN
p3,NaN,NaN,NaN,NaN,NaN
p4,NaN,NaN,NaN,NaN,2.0
p5,NaN,NaN,NaN,4.0,NaN
p6,NaN,NaN,NaN,NaN,4.0


In [168]:
test = pd.DataFrame(test,index=['p1','p2','p3','p4','p5','p6'],columns=['B1','B2','B3','B4','B5'])
test

,B1,B2,B3,B4,B5
p1,NaN,NaN,NaN,NaN,NaN
p2,NaN,3.0,5.0,NaN,NaN
p3,NaN,NaN,NaN,NaN,NaN
p4,NaN,NaN,NaN,NaN,2.0
p5,NaN,NaN,NaN,4.0,NaN
p6,NaN,NaN,NaN,NaN,4.0


# Building Raw average model 

In [159]:
# function to calculate mean of data frame 

def menaOfDf(df):
    count = df.count().sum()
    sum_total = df.sum().sum()
    mean = (sum_total/count)
    return mean


In [169]:
train_mean = menaOfDf(train)
train_mean

3.2999999999999998

In [161]:
#Creating a similar data frame with the existing one and fill with raw average value

def createAndFillAvg(df,val):
    df_predict= pd.DataFrame().reindex_like(df)
    df_predict= df_predict.fillna(val)
    return df_predict

train_predict = createAndFillAvg(train,train_mean)
test_predict  = createAndFillAvg(test,train_mean)

In [170]:
train_predict

,B1,B2,B3,B4,B5
p1,3.3,3.3,3.3,3.3,3.3
p2,3.3,3.3,3.3,3.3,3.3
p3,3.3,3.3,3.3,3.3,3.3
p4,3.3,3.3,3.3,3.3,3.3
p5,3.3,3.3,3.3,3.3,3.3
p6,3.3,3.3,3.3,3.3,3.3


## Evaluating Raw average model

In [175]:
# custom function to calculate rsme of 2 data frame
def rmse(df1,df2):
    se = ((df1-df2)**2).sum()
    se = se.sum()
    count = df1.count().sum()
    mse = (se/count)
    rmse = (mse)**0.5
    return rmse


In [176]:
rmse_train = rmse(train,train_predict)
rmse_test =  rmse(test,train_predict)

In [177]:
rmse_train

1.1000000000000001

In [178]:
rmse_test

1.063014581273465

## Rmse result of train and test data for Raw average model  
rmse_train = 1.1000000000000001
rmse_test = 1.063014581273465



# Building Baseline predictor model
## calculting book_bias

In [179]:
book_mean = train.mean()
book_mean

B1    3.8
B2    2.0
B3    4.0
B4    3.0
B5    3.4
dtype: float64

In [180]:
book_bias = book_mean - train_mean
book_bias

B1    0.5
B2   -1.3
B3    0.7
B4   -0.3
B5    0.1
dtype: float64

## calculating User_bias

In [181]:
user_mean = train.mean(axis=1)
user_mean

p1    4.333333
p2    4.000000
p3    3.000000
p4    2.000000
p5    4.000000
p6    3.500000
dtype: float64

In [182]:
user_bias = user_mean - train_mean
user_bias

p1    1.033333
p2    0.700000
p3   -0.300000
p4   -1.300000
p5    0.700000
p6    0.200000
dtype: float64

In [183]:
train_predict

,B1,B2,B3,B4,B5
p1,3.3,3.3,3.3,3.3,3.3
p2,3.3,3.3,3.3,3.3,3.3
p3,3.3,3.3,3.3,3.3,3.3
p4,3.3,3.3,3.3,3.3,3.3
p5,3.3,3.3,3.3,3.3,3.3
p6,3.3,3.3,3.3,3.3,3.3


## Adding book_bais and user_bais  to predict rating 

In [184]:
train_predict= train_predict.add(book_bias).add(user_bias,axis=0)
train_predict

,B1,B2,B3,B4,B5
p1,4.833333,3.033333,5.033333,4.033333,4.433333
p2,4.500000,2.700000,4.700000,3.700000,4.100000
p3,3.500000,1.700000,3.700000,2.700000,3.100000
p4,2.500000,0.700000,2.700000,1.700000,2.100000
p5,4.500000,2.700000,4.700000,3.700000,4.100000
p6,4.000000,2.200000,4.200000,3.200000,3.600000


## Rounding of any  rating >5  to 5 and  less than 1 to 1 

In [185]:
def roundingNumber(df):
    df[df>5] =5
    df[df<1]=1
    return df


In [186]:
train_predict = roundingNumber(train_predict)
train_predict

,B1,B2,B3,B4,B5
p1,4.833333,3.033333,5.0,4.033333,4.433333
p2,4.500000,2.700000,4.7,3.700000,4.100000
p3,3.500000,1.700000,3.7,2.700000,3.100000
p4,2.500000,1.000000,2.7,1.700000,2.100000
p5,4.500000,2.700000,4.7,3.700000,4.100000
p6,4.000000,2.200000,4.2,3.200000,3.600000


## Evaluation of Baseline predictor model  for training data

In [187]:
rsme_base_line_train = rsme(train,train_predict)
rsme_base_line_train

0.50870205206759067

## Evaluation of Baseline predictor model for test data

In [188]:
rsme_base_line_test = rsme(test,train_predict)
rsme_base_line_test

0.29664793948382639

## Performace improvement  

In [104]:
def improveInPerformance(df1, df2):
    return ((df1 - df2)/df1) * 100


In [189]:
improvement_train = improveInPerformance(rmse_train, rsme_base_line_train)
improvement_train

53.754358902946308

In [190]:
improvement_test = improveInPerformance(rmse_test, rsme_base_line_test)
improvement_test 

72.093709276457005


# Conclusion 

###   Baseline predictor shows much better results than the simple raw average model. However, to improve this model, we can use collaborative filtering  (item based or user based) with a large volume of data set. As we can clearly see that Base line predictor is more reliable than Raw average , but result may vary with diffrent data set and high volume data. A larger dataset is recommended to perform the computation.
    

